## 1. 安装和导入

首先需要解决NumPy版本问题，然后安装PyQula

In [ ]:
# 检查NumPy版本
import numpy as np
print(f"NumPy version: {np.__version__}")

# 如果是2.x，需要降级：
# 以管理员身份运行: conda install 'numpy<2.0' scipy -y

if np.__version__.startswith('2.'):
    print("\n⚠️  警告: NumPy 2.x 与 SciPy 不兼容")
    print("请运行: conda install 'numpy<2.0' scipy -y")
else:
    print("✓ NumPy版本正确")

In [ ]:
# 安装PyQula (如果尚未安装)
# !pip install git+https://github.com/joselado/pyqula

# 导入库
import sys
sys.path.append('../code')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

# 尝试导入PyQula
try:
    import pyqula
    from tb_model_pyqula import MnBi2Te4_PyQula
    PYQULA_AVAILABLE = True
    print("✓ PyQula imported successfully")
except ImportError as e:
    PYQULA_AVAILABLE = False
    print(f"✗ PyQula not available: {e}")
    print("\nInstall with:")
    print("  pip install git+https://github.com/joselado/pyqula")

## 2. 创建 MnBi₂Te₄ 模型

### 物理参数

- **晶格常数** $a = 4.38$ Å
- **最近邻跳跃** $t = 1.0$ eV  
- **自旋轨道耦合** $\lambda_{\text{SO}} = 0.3$ eV
- **交换场** $M = 0.5$ eV（反铁磁）
- **层间耦合** $t_\perp = 0.2$ eV

In [ ]:
if PYQULA_AVAILABLE:
    # 创建模型
    model = MnBi2Te4_PyQula(
        a=4.38,          # 晶格常数 (Å)
        t=1.0,           # 跳跃积分 (eV)
        lambda_SO=0.3,   # SOC强度 (eV)
        M=0.5,           # 交换场 (eV)
        t_perp=0.2       # 层间耦合 (eV)
    )
    
    print("Model created with parameters:")
    print(f"  a = {model.a} Å")
    print(f"  t = {model.t} eV")
    print(f"  λ_SO = {model.lambda_SO} eV")
    print(f"  M = {model.M} eV")
    print(f"  t_⊥ = {model.t_perp} eV")
else:
    print("PyQula not available. Please install first.")

## 3. 能带结构计算

计算布里渊区高对称路径上的能带：

$$\Gamma \to M \to K \to \Gamma$$

In [ ]:
if PYQULA_AVAILABLE:
    # 计算能带
    print("Calculating band structure...")
    k_path, bands = model.calculate_bands(nk=200)
    
    # 绘制能带
    plt.figure(figsize=(8, 6))
    
    # 绘制所有能带
    for i in range(bands.shape[1]):
        plt.plot(k_path, bands[:, i], 'b-', alpha=0.6, linewidth=1.5)
    
    # 标记高对称点
    nk = len(k_path)
    plt.axvline(0, color='k', linestyle='--', alpha=0.3)
    plt.axvline(nk//3, color='k', linestyle='--', alpha=0.3)
    plt.axvline(2*nk//3, color='k', linestyle='--', alpha=0.3)
    plt.axvline(nk-1, color='k', linestyle='--', alpha=0.3)
    
    plt.xticks([0, nk//3, 2*nk//3, nk-1], ['Γ', 'M', 'K', 'Γ'])
    plt.ylabel('Energy (eV)', fontsize=12)
    plt.xlabel('k-path', fontsize=12)
    plt.title('MnBi₂Te₄ Band Structure', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.axhline(0, color='r', linestyle='-', alpha=0.5, linewidth=0.8, label='E_F')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f"\nNumber of bands: {bands.shape[1]}")
    print(f"Energy range: [{bands.min():.3f}, {bands.max():.3f}] eV")

## 4. Berry 曲率计算

### Kubo 公式

$$\Omega_n(\mathbf{k}) = -2\text{Im}\sum_{m\neq n} \frac{\langle n|v_x|m\rangle \langle m|v_y|n\rangle}{(E_n - E_m)^2}$$

其中速度算符：
$$v_\mu = \frac{1}{\hbar}\frac{\partial H}{\partial k_\mu}$$

In [ ]:
if PYQULA_AVAILABLE:
    # 计算Berry曲率（在BZ网格上）
    print("Calculating Berry curvature...")
    
    nk = 50  # k空间网格
    kx = np.linspace(-np.pi/model.a, np.pi/model.a, nk)
    ky = np.linspace(-np.pi/model.a, np.pi/model.a, nk)
    
    KX, KY = np.meshgrid(kx, ky)
    berry_map = np.zeros_like(KX)
    
    # 计算每个k点的Berry曲率（最低导带）
    for i in range(nk):
        for j in range(nk):
            try:
                omega = model.calculate_berry_curvature_kubo(kx[i], ky[j], band_index=0)
                berry_map[j, i] = omega
            except:
                berry_map[j, i] = 0.0
        
        if (i+1) % 10 == 0:
            print(f"  Progress: {i+1}/{nk}")
    
    print("\nBerry curvature statistics:")
    print(f"  Max: {berry_map.max():.6f}")
    print(f"  Min: {berry_map.min():.6f}")
    print(f"  Mean: {berry_map.mean():.6f}")

In [ ]:
if PYQULA_AVAILABLE:
    # 绘制Berry曲率分布
    plt.figure(figsize=(10, 8))
    
    # 使用对称色标
    vmax = max(abs(berry_map.min()), abs(berry_map.max()))
    
    plt.pcolormesh(KX, KY, berry_map, 
                   cmap='RdBu_r', 
                   shading='auto',
                   vmin=-vmax, vmax=vmax)
    
    cbar = plt.colorbar(label='Ω(k) (Å²)', pad=0.02)
    cbar.ax.tick_params(labelsize=10)
    
    plt.xlabel('$k_x$ (Å⁻¹)', fontsize=12)
    plt.ylabel('$k_y$ (Å⁻¹)', fontsize=12)
    plt.title('Berry Curvature Distribution', fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    # 计算Chern数
    chern = berry_map.sum() * (kx[1]-kx[0]) * (ky[1]-ky[0]) / (2*np.pi)
    print(f"\nChern number: {chern:.4f}")

## 5. 层霍尔电导率

### 公式

内禀贡献（Berry曲率差）：

$$\sigma_{xy}^{\text{layer}} = -\frac{e^2}{\hbar} \int_{\text{BZ}} \frac{d^2k}{(2\pi)^2} f(E_n(\mathbf{k})) \Delta\Omega_n(\mathbf{k})$$

其中：
$$\Delta\Omega_n = \Omega_n^{(1)} - \Omega_n^{(2)}$$

量子度规贡献（非线性）：
$$\sigma_{xy}^{\text{metric}} \propto \int g_n^{xy} \cdot \text{DOS}$$

In [ ]:
if PYQULA_AVAILABLE:
    # 计算层霍尔电导（温度扫描）
    print("Calculating layer Hall conductivity vs temperature...")
    
    temperatures = np.linspace(1, 300, 30)  # K
    sigma_layer = []
    
    for T in temperatures:
        try:
            sigma = model.calculate_layer_hall_conductivity(
                temperature=T,
                nk=40,
                eta=1e-6
            )
            sigma_layer.append(sigma)
        except Exception as e:
            print(f"  Error at T={T}K: {e}")
            sigma_layer.append(0.0)
    
    sigma_layer = np.array(sigma_layer)
    
    # 绘制结果
    plt.figure(figsize=(10, 6))
    plt.plot(temperatures, sigma_layer * 1e3, 'o-', linewidth=2, markersize=6)
    plt.xlabel('Temperature (K)', fontsize=12)
    plt.ylabel('$\\sigma_{xy}^{\\text{layer}}$ ($10^{-3} e^2/h$)', fontsize=12)
    plt.title('Layer Hall Conductivity vs Temperature', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nAt T=10K: σ_layer = {sigma_layer[9]*1e3:.6f} × 10⁻³ e²/h")
    print(f"At T=300K: σ_layer = {sigma_layer[-1]*1e3:.6f} × 10⁻³ e²/h")

## 6. 参数依赖性研究

研究层霍尔效应对关键参数的依赖

In [ ]:
if PYQULA_AVAILABLE:
    # 扫描交换场M
    print("Scanning exchange field M...")
    
    M_values = np.linspace(0.1, 1.0, 10)
    sigma_vs_M = []
    
    for M in M_values:
        model_temp = MnBi2Te4_PyQula(
            a=4.38, t=1.0, lambda_SO=0.3, 
            M=M, t_perp=0.2
        )
        
        try:
            sigma = model_temp.calculate_layer_hall_conductivity(
                temperature=10, nk=30, eta=1e-6
            )
            sigma_vs_M.append(sigma)
        except:
            sigma_vs_M.append(0.0)
    
    # 扫描SOC强度
    print("Scanning SOC strength λ...")
    
    lambda_values = np.linspace(0.1, 0.5, 10)
    sigma_vs_lambda = []
    
    for lam in lambda_values:
        model_temp = MnBi2Te4_PyQula(
            a=4.38, t=1.0, lambda_SO=lam,
            M=0.5, t_perp=0.2
        )
        
        try:
            sigma = model_temp.calculate_layer_hall_conductivity(
                temperature=10, nk=30, eta=1e-6
            )
            sigma_vs_lambda.append(sigma)
        except:
            sigma_vs_lambda.append(0.0)
    
    # 绘制
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(M_values, np.array(sigma_vs_M)*1e3, 'o-', linewidth=2, markersize=8)
    axes[0].set_xlabel('Exchange Field M (eV)', fontsize=12)
    axes[0].set_ylabel('$\\sigma_{xy}^{\\text{layer}}$ ($10^{-3} e^2/h$)', fontsize=12)
    axes[0].set_title('Layer Hall vs M', fontsize=13, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(lambda_values, np.array(sigma_vs_lambda)*1e3, 's-', 
                 linewidth=2, markersize=8, color='C1')
    axes[1].set_xlabel('SOC Strength λ (eV)', fontsize=12)
    axes[1].set_ylabel('$\\sigma_{xy}^{\\text{layer}}$ ($10^{-3} e^2/h$)', fontsize=12)
    axes[1].set_title('Layer Hall vs λ_SO', fontsize=13, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nParameter scan complete!")

## 7. 与文献对比

**参考文献：**
- Chen et al., arxiv:2501.xxxxx (2025)
- Gao et al., Nature 595, 521 (2021)

预期结果：
- $\sigma_{xy}^{\text{layer}} \sim 10^{-3} - 10^{-2}$ $e^2/h$
- Berry曲率在K点附近最强
- 随温度降低而增强

In [ ]:
# 总结结果
if PYQULA_AVAILABLE:
    print("="*60)
    print("SUMMARY OF CALCULATIONS")
    print("="*60)
    print(f"\n1. Model parameters:")
    print(f"   a = {model.a} Å, t = {model.t} eV")
    print(f"   λ_SO = {model.lambda_SO} eV, M = {model.M} eV")
    print(f"\n2. Band structure:")
    print(f"   {bands.shape[1]} bands calculated")
    print(f"   Energy gap: ~{abs(bands[nk//2, 2] - bands[nk//2, 3]):.3f} eV")
    print(f"\n3. Berry curvature:")
    print(f"   Max |Ω| = {abs(berry_map).max():.6f} Å²")
    print(f"   Chern number C ≈ {chern:.2f}")
    print(f"\n4. Layer Hall conductivity:")
    print(f"   T=10K: σ = {sigma_layer[9]*1e3:.6f} × 10⁻³ e²/h")
    print(f"   T=300K: σ = {sigma_layer[-1]*1e3:.6f} × 10⁻³ e²/h")
    print("\n" + "="*60)
else:
    print("PyQula not available. Install to run calculations.")

## 下一步

1. **实验设计**：参见 `03_hall_bar_design.ipynb`
2. **Kwant实现**：输运计算和散射
3. **无序效应**：杂质散射对层霍尔的影响
4. **温度依赖**：更精细的费米分布
5. **文献验证**：与实验数据对比